In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
      super().__init__()
      self.linear_stack = nn.Sequential(
          nn.Linear(15, 10),
          nn.LeakyReLU(),
          nn.Linear(10, 5),
          nn.LeakyReLU(),
          nn.Linear(5, 1),
          nn.LeakyReLU()
      )

  def forward(self, x):
    logits = self.linear_stack(x)
    return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

NeuralNetwork(
  (linear_stack): Sequential(
    (0): Linear(in_features=15, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=5, out_features=1, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
)


In [5]:
def train_one_epoch(epoch_index):
  running_loss = 0.
  last_loss = 0.

  for i, data in enumerate(training_loader):
    # Every data instance is an input + label pair
    inputs, labels = data

    # Zero gradients for every batch
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = model(inputs)

    # Compute the loss and its gradients
    loss = loss_fn(outputs, labels)
    loss.backward()

    # Adjust learning weights
    optimizer.step()

    # Gather data and report
    running_loss += loss.item()
    if i % 1000 == 999:
      last_loss = running_loss / 1000 # loss per batch
      print('  batch {} loss: {}'.format(i + 1, last_loss))
      running_loss = 0.

  return last_loss

In [8]:
epoch_number = 0
EPOCHS = 5

average_losses = []

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(epoch_number)
    average_losses.append(avg_loss)

    #   running_vloss = 0.0

    model.eval()

    #   with torch.no_grad():
    #     for i, vdata in enumerate(testing_loader):
    #       vinputs, vlabels = vdata
    #       voutputs = model(vinputs)
    #       vloss = loss_fn(voutputs, vlabels)
    #       running_vloss += vloss

    #   avg_vloss = running_vloss / (i + 1)
    print('LOSS train {}'.format(avg_loss))

    model_path = 'model_{}'.format(epoch_number)
    torch.save(model.state_dict(), model_path)

EPOCH 1:


NameError: name 'training_loader' is not defined

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, dataset_file):
    pass

  def __len__(self):
    pass

  def __getitem__(self):
    pass

In [13]:
def data_prep(filename):
  desired_columns = ("id",
                     "pre_loans5",
                     "pre_loans530",
                     "pre_loans3060",
                     "pre_loans6090",
                     "pre_loans90",
                     "pre_till_pclose",
                     "pre_till_fclose",
                     "pre_loans_credit_limit",
                     "pre_loans_next_pay_summ",
                     "pre_loans_outstanding",
                     "pre_loans_total_overdue",
                     "pre_loans_max_overdue_sum",
                     "pre_util",
                     "pre_over2limit",
                     "pre_maxover2limit")
  data = pd.read_csv(filename,usecols = desired_columns)
  return data